# Installing Necessary Libraries

In [6]:
from openai import OpenAI
import csv
import json
import random

# Defining my client key

In [8]:
client = OpenAI(api_key="your api key")

# Loading the dataset

In [11]:
# load csv

comment_list = []
response_list = []

with open(r"C:\Users\mahes\Downloads\archive (2)\reddit_text-davinci-002.csv",encoding='utf-8') as file:
    file = csv.reader(file)
    
    # read file line by line
    for line in file:
        # skip first line
        if line[0]=='Comment':
            continue
            
        # append comments and responses to respective lists
        comment_list.append(line[0])
        response_list.append(line[1])

In [8]:
len(comment_list)

3180

# Formatting data 

In [12]:
# construct training examples
example_list = []

intstructions_string_few_shot = """Wisp is a compassionate mental health counselor offering support and guidance to [someone struggling with mental illness]. 
How would you encourage them to prioritize self-care, seek professional help, and stay positive during their journey towards healing and recovery?
Here are examples of ShawGPT responding to viewer comments.

Client: "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
How can I change my feeling of being worthless to everyone?"

Wisp: Heck, sure thing, hun!Feelings of 'depression' have a deeply-rooted base in physical structures that may not be functioning very well at present; and, we can certainly turn them on again using means that you are able to find around the house and with relative ease :)After that, emotional and spiritual support will be liberally applied.


Client: "I have so many issues to address. I have a history of sexual abuse, Iâ€™m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and Iâ€™m beginning to have anxiety. I have low self esteem but Iâ€™ve been happily married for almost 35 years.
   Iâ€™ve never had counseling about any of this. Do I have too many issues to address in counseling?"

Wisp: Absolutely not! It is never too much. The most important thing is that you are reaching out to get help. Â Therapy helps you to develop healthier coping strategies and that can help reduce the anxiety and depression as well as improve your sleep. This can all be done at a pace that is best for you. Your therapist can help you process all of this in a safe and supportive space.


Client: I struggle with depression as well as pretty intense mood swings throughout the month. I experience highs where I feel amazing and energetic and then lows where I lack focus, energy, and generally have a more dark outlook on my life. How can I live a more balanced life?

Wisp: It's fun to ride the roller coaster from time to time, isn't it? :)But, it's also weary-making, and leads to drainage that no man or woman can hardly anticipate!Balance comes with proper understanding of the different bodies you possess and how they function.And, to begin, we focus upon your physical, and move right up the latter to the spiritual, and begin cleaning you out.Unbalance is an experience of blocked energies that should be naturally flowing (call them what thou mayest); when blockages are removed, what is naturally there flows, and flows beautifully...
"""

for i in range(len(comment_list)):    
    system_dict = {"role": "system", "content": intstructions_string_few_shot}
    user_dict = {"role": "user", "content": comment_list[i]}
    assistant_dict = {"role": "assistant", "content": response_list[i]}
    
    messages_list = [system_dict, user_dict, assistant_dict]
    
    example_list.append({"messages": messages_list})

# Splitting into training and testing

In [10]:
# create train/validation split
validation_index_list = random.sample(range(0, len(example_list)-1), 700)

validation_data_list = [example_list[index] for index in validation_index_list]

for example in validation_data_list:
    example_list.remove(example)

# Creating json files for the datasets

In [11]:
# write examples to file
with open('data/training-data.jsonl', 'w') as training_file:
    for example in example_list:
        json.dump(example, training_file)
        training_file.write('\n')

with open('data/validation-data.jsonl', 'w') as validation_file:
    for example in validation_data_list:
        json.dump(example, validation_file)
        validation_file.write('\n')

In [12]:
training_file = client.files.create(
  file = open("data/training-data.jsonl", "rb"),
  purpose = "fine-tune"
)

validation_file = client.files.create(
  file = open("data/validation-data.jsonl", "rb"),
  purpose = "fine-tune"
)

# Creating a model and fini tuning it

In [13]:
client.fine_tuning.jobs.create(
    training_file = training_file.id,
    validation_file = validation_file.id,
    suffix = "Wisp",
    model = "gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-GDdU9auaPAHttlKHt2ZDrkzW', created_at=1710172021, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-kHhHEN2bEHNy1LX3kVXNHGkg', result_files=[], status='validating_files', trained_tokens=None, training_file='file-T0ZnwRGGZ9Mx0KCbgl1pAGmT', validation_file='file-cVnRcEeKv5jDKaiiLEVaDHo6', user_provided_suffix='Wisp')

# Testing the model

In [13]:
test_comment = "Hi! Iam feeling very depressed and sad."

In [15]:
response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:personal:wisp:91dhHss7",
    messages=[
    {"role": "system", "content": intstructions_string_few_shot},
    {"role": "user", "content": test_comment}
    ]
)

In [16]:
print(dict(response)['choices'][0].message.content)

Therapy can help you when you're feeling depressed and sad. You don't have to do this alone.
